In [2]:
!pip install eli5

     |████████████████████████████████| 112kB 2.9MB/s 


In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

In [5]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix"

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [0]:
#ls data
df = pd.read_csv("data/men_shoes.csv", low_memory=False)

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
  X = df[ feats ].values
  y = df['prices_amountmin'].values

  scores = cross_val_score(model, X, y, scoring="neg_mean_absolute_error")
  return np.mean(scores), np.std(scores)

In [8]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]
run_model( [ 'brand_cat' ] )

(-58.133398968282776, 4.206122611474276)

In [9]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state= False)
run_model( [ 'brand_cat' ], model)

(-57.31783843165656, 4.181246596160967)

In [122]:
#mozemy lepiej poznać nasze dane 
df.head()

,id,asins,brand,categories,colors,count,dateadded,dateupdated,descriptions,dimension,ean,features,flavors,imageurls,isbn,keys,manufacturer,manufacturernumber,merchants,name,prices_amountmin,prices_amountmax,prices_availability,prices_color,prices_condition,prices_count,prices_currency,prices_dateadded,prices_dateseen,prices_flavor,prices_issale,prices_merchant,prices_offer,prices_returnpolicy,prices_shipping,prices_size,prices_source,prices_sourceurls,prices_warranty,quantities,...,feat_retail price,feat_sub type,feat_seller inventory number,feat_contains batteries,feat_main color,feat_athlete,feat_auto,feat_shoe closure,feat_purpose,feat_watch case shape,feat_garment care,feat_chest,feat_fabric care,feat_closure style,feat_stock no.,feat_kids backpacks,feat_country of origin assembly:,feat_number of heat settings,feat_part type,feat_frame type,feat_fabric,feat_year made,feat_bag size range,feat_batteries required?,feat_age end,feat_dial material,feat_ring style,feat_band length,feat_hat style,feat_heat zones,feat_product type,feat_animal type,feat_brand_cat,feat_color_cat,feat_gender_cat,feat_manufacturer part number_cat,feat_material_cat,feat_sport_cat,feat_style_cat,feat_animal type_cat
0,AVpfHrJ6ilAPnD_xVXOI,NaN,josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",39.89,39.89,NaN,NaN,NaN,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,True,NaN,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,-1,-1,-1,-1
1,AVpfHrJ6ilAPnD_xVXOI,NaN,josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",51.99,51.99,NaN,NaN,new,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,False,UnbeatableSale - Walmart.com,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,-1,-1,-1,-1
2,AVpfHsWP1cnluZ0-eVZ7,NaN,servus by honeywell,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:57Z""],""name"":""...",Servus By Honeywell Shoe Studs Zsr101blmlg,40.02,40.02,NaN,NaN,new,NaN,USD,2016-06-14T04:29:57Z,2016-03-08T00:00:00Z,NaN,False,SIM Supply Inc - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,0,-1,-1,-1,-1,-1
3,AVpfHsWP1cnluZ0-eVZ7,NaN,servus by honeywell,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://

In [13]:
df["features"].values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       ...,
       '[{"key":"Gender","value":["Unisex"]},{"key":"Frame Style","value":["Wrap"]},{"key":"Polarized Lenses","value":["Polarized lenses"]},{"key":"Color","value":["Multi-color"]}]',


In [18]:
str_dict = '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]'
literal_eval(str_dict)[0]['value'][0]

'Men'

# na wejsciu:
[
 {'key': 'Gender', 'value': ['Men']},<br>
 {'key': 'Shoe Size', 'value': ['M']},<br>
 {'key': 'Shoe Category', 'value': ["Men's Shoes"]},<br>
 {'key': 'Color', 'value': ['Multicolor']},<br>
 {'key': 'Manufacturer Part Number', 'value': ['8190-W-NAVY-7.5']}<br>
 {'key': 'Brand', 'value': ['Josmo']} ]

 #co chcemy mieć :
 <br>{
     'Gender': 'Men',<br>
    'Shoe size': 'M'
 }

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan' : return output_dict

  features = literal_eval(x.replace('\\"', '"'))
  for item in features:
    #strip,lower - małymi lierami i bez spacji
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()

    output_dict[key] = value

  return output_dict

df['features_parsed'] = df['features'].map(parse_features)

In [34]:
#df['features_parsed'].head().values
#df.features_parsed.head().values
keys = set()

df['features_parsed'].map( lambda x: keys.update(x.keys()))
len(keys)

476

In [109]:
def get_name_feat(key):
  return 'feat_'+key

for key in tqdm_notebook(keys):
   df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)

In [39]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_bag size range', 'feat_batteries required?', 'feat_age end',
       'feat_dial material', 'feat_ring style', 'feat_band length',
       'feat_hat style', 'feat_heat zones', 'feat_product type',
       'feat_animal type'],
      dtype='object', length=526)

In [0]:
#badamy udział cechy w całości
keys_stat = {}
for key in keys:
  keys_stat[key] = df[ False == df[get_name_feat(key)].isnull() ].shape[0] / df.shape[0] * 100

In [53]:
#keys_stat
{k:v for k,v in keys_stat.items() if v > 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
#feature engineering
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state= False)


In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]

df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]

#automatyzacja dodawania kolumn / ich nazw - jak wyzej
for key in keys:
  df[get_name_feat(key) + '_cat'] =  df[get_name_feat(key)].factorize()[0]


In [67]:
#df[ df.brand == df.feat_brand].shape
#df[ df.brand == df.feat_brand] [[ 'brand', 'feat_brand']].head()

df['brand'] = df['brand'].map(lambda x: str(x).lower())
df[ df.brand == df.feat_brand].shape


(8846, 531)

In [0]:

feats = ['']

In [71]:
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model( [ 'brand_cat'],model )


(-57.30156868886563, 4.225904109891617)

In [133]:
feats_cat = [x for x  in df.columns if 'cat' in x]
feats_cat

['categories',
 'brand_cat',
 'feat_catalog',
 'feat_fabrication',
 'feat_certifications and listings',
 'feat_recommended location',
 'feat_shoe category',
 'feat_clothing category',
 'feat_multi pack indicator',
 'feat_location - city/state',
 'feat_location - country',
 'feat_brand_cat',
 'feat_color_cat',
 'feat_gender_cat',
 'feat_manufacturer part number_cat',
 'feat_material_cat',
 'feat_sport_cat',
 'feat_style_cat',
 'feat_animal type_cat',
 'feat_age group_cat',
 'feat_era_cat',
 'feat_arm_cat',
 'feat_waist_cat',
 'feat_walmart no._cat',
 'feat_age start_cat',
 'feat_metal stamp_cat',
 'feat_fabric care instructions_cat',
 'feat_designed for_cat',
 'feat_target audience_cat',
 'feat_very popular bag now a days_cat',
 'feat_protection_cat',
 'feat_stability_cat',
 'feat_protects against_cat',
 'feat_net weight_cat',
 'feat_occasion_cat',
 'feat_leg opening:_cat',
 'feat_catalog_cat',
 'feat_fits model_cat',
 'feat_frame style_cat',
 'feat_assembled in country of origin_cat',


In [118]:
df['weight'].unique()

array([nan, '3.0 lbs', '9 g', '1.45 lbs', '0.45 lbs', '1.0 lbs',
       '0.23 lbs', '5.0 lbs', '5.5 lbs', '7.45 lbs', '4.0 lbs',
       '2.7969 lbs', '3.9 lbs', '4.6 pounds', '2.1 lbs', '1.1057 lbs',
       '15.0 lbs', '2.4 ounces', '454 g', '0.105 lbs', '9.1 ounces',
       '4.8 lbs', '6.1 lbs', '6.5 lbs', '1.1041 lbs', '1.3 Kg', '91 g',
       '20.0 lbs', '6.0 lbs', '386 g', '0.81 lbs', '4.5 lbs',
       '0.5 ounces', '2.0 lbs', '3.13 lbs', '5.9 lbs', '6.15 lbs',
       '1 pounds', '1.95 lbs', '2.15 lbs', '2 pounds', '2.1 pounds',
       '14 Kg', '0.4788 lbs', '10.0 lbs', '0.38 lbs', '2.5 lbs',
       '68.912 lbs', '45 g', '13.09 lbs', '2.5 pounds', '0.21 lbs',
       '16.75 lbs', '6.3 lbs', '272 g', '1.8 Kg', '2.8 pounds', '0.1 lbs',
       '5.05 lbs', '0.28 lbs', '76.08 lbs', '0.15 lbs', '200 g',
       '7.8 pounds', '399 g', '4.95 lbs', '64.144 lbs', '24 pounds',
       '73.696 lbs', '1.6 lbs', '6.6 ounces', '5 g', '1.2 Kg', '862 g',
       '3.05 lb', '8.6 ounces', '3.6 lbs', '71.

In [0]:
feats = [ 'brand_cat','feat_gender_cat','feat_material_cat','feat_movement_cat','feat_style_cat','feat_shape_cat']
#feats += feats_cat
# lista unikalnych wartosci
#feats = list(set(feats))
#'feat_sport_cat','feat_style_cat' 'feat_eye size:_cat' 'feat_age_cat', ,'feat_occasion_cat',
#-------------------
#feats +=[]

model = RandomForestRegressor(max_depth=5, n_estimators=100)
result = run_model( feats,model )

In [204]:
import eli5
print(result)
X = df[feats].values
y = df['prices_amountmin'].values
m = RandomForestRegressor(max_depth=5, n_estimators=100)
m.fit(X,y)

perm = PermutationImportance(m, random_state=1).fit(X,y)
eli5.show_weights(perm, feature_names=feats)


(-57.130181823119074, 4.178357948893857)


Weight,Feature
0.2635 ± 0.0102,brand_cat
0.1086 ± 0.0052,feat_material_cat
0.0278 ± 0.0019,feat_gender_cat
0.0182 ± 0.0010,feat_movement_cat
0.0051 ± 0.0016,feat_style_cat
0.0007 ± 0.0000,feat_shape_cat


In [205]:
df['brand'].value_counts()

#df[df['brand'] == 'nike'].features_parsed.head().values
df[df['brand'] == 'nike'].features_parsed.sample(5).values

array([{}, {'style': 'athletic sneakers', 'condition': 'new with box'},
       {'style': 'athletic sneakers', 'country/region of manufacture': 'china', 'condition': 'new with box'},
       {'style': 'n/a', 'lens technology': 'no,no,59,6', 'mpn': 'ev0674-420-202', 'gender': 'men', 'manufacturer': 'nike', 'shipping weight (in pounds)': '1.0', 'manufacturer part number': 'ev0674-420-202', 'brand': 'nike', 'age group': 'adult'},
       {}], dtype=object)

In [209]:
cd matrix_one/

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_one


In [210]:
!git add matrix_one/day5.ipynb

fatal: pathspec 'matrix_one/day5.ipynb' did not match any files
